In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt

In [ ]:
import keras
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from keras import optimizers
import time

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
df1=pd.read_csv("/kaggle/input/bitstamp-btc-stock-time-series/bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv")

In [ ]:
df1.head()

In [ ]:
df1.shape

To find out the null values

In [ ]:
sns.heatmap(df1.isnull(),yticklabels=False)

In [ ]:
df1.dropna(how='any',inplace=True)#drops rows who have all of their inputs as NaN 

In [ ]:
df1.shape

In [ ]:
df1.head()

In [ ]:
plt.plot(df1['Close'], color="black")
plt.show()

In [ ]:
df1.dtypes

In [ ]:
#first =df1["Timestamp"][0]

In [ ]:
#from datetime import datetime
#dt_obj = datetime.fromtimestamp(first)
  
#print("date_time:",dt_obj)
#print("type of dt:",type(dt_obj))

In [ ]:

%matplotlib inline 
plt.gcf().set_size_inches(20, 10, forward=True)
plt.plot(df1['Weighted_Price'], color="black")

plt.title(" Share Price")
plt.xlabel("Days")
plt.ylabel("Share Price")

plt.show()


In [ ]:
#pd.to_datetime(df1['Timestamp']).apply(lambda x: x.date())

In [ ]:
#df2=df1
#pd.to_datetime(df2['Timestamp']).dt.date

In [ ]:
#df1.head()

In [ ]:
#df2.head()

In [ ]:
df1["Date"]=df1.Timestamp.apply(lambda x: pd.datetime.fromtimestamp(x).date())

In [ ]:
df1.head()

In [ ]:
df1.drop(['Timestamp'], axis = 1,inplace=True)

In [ ]:
df1.head()

In [ ]:
df1.set_index('Date', inplace=True)
df1.info()

In [ ]:
%matplotlib inline 
plt.gcf().set_size_inches(20, 10, forward=True)
plt.plot(df1['Weighted_Price'], color="red")

plt.title(" Share Price")
plt.xlabel("Days")
plt.ylabel("Share Price")

plt.show()


In [ ]:
df2=df1.iloc[2458600:,:]

In [ ]:
%matplotlib inline 
plt.gcf().set_size_inches(20, 10, forward=True)
plt.plot(df2['Weighted_Price'], color="red")

plt.title(" Share Price")
plt.xlabel("Days")
plt.ylabel("Share Price")

plt.show()


time series analysis

In [ ]:
from sklearn import preprocessing
data_normaliser = preprocessing.StandardScaler()
df1_normalised = data_normaliser.fit_transform(df2)

In [ ]:
df1_normalised[0,:]

In [ ]:
history_points=60
 

In [ ]:
ohlcv_histories_normalised = np.array([df1_normalised[i  : i + history_points].copy() for i in range(len(df1_normalised) - history_points)])

In [ ]:
weighted_avg_values_normalised = np.array([df1_normalised[:,-1][i + history_points].copy() for i in range(len(df1_normalised) - history_points)])

In [ ]:
print(weighted_avg_values_normalised.shape)
weighted_avg_values_normalised = np.expand_dims(weighted_avg_values_normalised, -1)
weighted_avg_values_normalised.shape

In [ ]:
weighted_avg_values = np.array([df2["Weighted_Price"][i + history_points].copy() for i in range(len(df2) - history_points)])

In [ ]:
y_normaliser = preprocessing.StandardScaler()
y_normaliser.fit(np.expand_dims( weighted_avg_values,-1 ))

In [ ]:
print(weighted_avg_values.shape)

In [ ]:
test_split = 0.90 # the percent of data to be used for testing
n = int(ohlcv_histories_normalised.shape[0] * test_split)
n

In [ ]:
x_train = ohlcv_histories_normalised[:n]
print(x_train.shape)

In [ ]:
y_train = weighted_avg_values_normalised[:n]
y_train.reshape(n,1)
print(y_train.shape)

In [ ]:
x_test = ohlcv_histories_normalised[n:]
y_test = weighted_avg_values_normalised[n:]

In [ ]:
corrMatx =  df1.corr()
corrMatx

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

In [ ]:
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
with tpu_strategy.scope():
    start = time.process_time()
    model21=Sequential()
    model21.add(LSTM(units=100, return_sequences=True, input_shape=(history_points,7)))
    model21.add(Dropout(0.20))
    model21.add(LSTM(units=100, return_sequences=True))
    model21.add(Dropout(0.20)) 
    model21.add(LSTM(units=100, return_sequences=True))
    model21.add(Dropout(0.20)) 
    model21.add(LSTM(units=100))
    model21.add(Dropout(0.20)) 
    model21.add(Dense(units=1))
    model21.compile(optimizer="adam", loss="mean_squared_error")



In [ ]:
model21.fit(x_train,y_train,batch_size=128, epochs=12) 
print(time.process_time() - start)

In [ ]:
print(model21.summary())

In [ ]:
y_test_predicted21 = model21.predict(x_test)

In [ ]:
y_test= y_normaliser.inverse_transform(y_test)
y_test

In [ ]:
y_test_predicted21 = y_normaliser.inverse_transform(y_test_predicted21)
y_test_predicted21

In [ ]:
plt.gcf().set_size_inches(22, 10, forward=True)

start = 0
end = -1
#actual=plt.plot(actual[start+60:end], label= "actual")
real = plt.plot(y_test[start:end], label='real', color="black")
pred = plt.plot(y_test_predicted21[start:end], label='predicted', color="red")

plt.legend(['Real', 'Predicted'])
plt.title(" Share Price")
plt.xlabel("Days")
plt.ylabel("Share Price")
plt.show()